In [34]:
# Flow_from_directory
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.regularizers import L2
from keras.applications import resnet_v2
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam


In [35]:
IMAGE_SIZE = 224

# Train data with validation split with data augmentation
train_generator =  ImageDataGenerator(preprocessing_function= resnet_v2.preprocess_input)



train= train_generator.flow_from_directory('data/full/calc/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=32, subset='training', shuffle=False)
val= train_generator.flow_from_directory('data/full/calc/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=32, subset='validation', shuffle=False)

# Test data
test_generator =ImageDataGenerator(preprocessing_function= resnet_v2.preprocess_input)
test = test_generator.flow_from_directory('data/full/calc/test/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=32,)

Found 988 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


In [36]:
pretrained_model = resnet_v2.ResNet50V2()

# wtf
pretrained_model.trainable = False
for layer in pretrained_model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True


model_layers = [
    pretrained_model,
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax'),
]

model = Sequential(model_layers)

In [37]:
model.compile(Adam(lr=0.01), loss=BinaryCrossentropy(label_smoothing=0.25), weighted_metrics=['accuracy'])

In [ ]:
model.fit(train, validation_data=val, epochs=30)

In [ ]:
# Plot the training and validation accuracy and loss at each epoch
import matplotlib.pyplot as plt
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

In [ ]:
# Create roc curve
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Get the predictions from the model using the generator
predictions = model.predict(test)

# Get the true values from the generator
true_values = test.classes

# Get the predicted probabilities for each class
pred_probabilities = predictions

# Get the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(true_values, pred_probabilities)

# Plot the ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='ResNet152')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')